In [1]:
import numpy as np
import pandas as pd
import os
import yfinance as yf
from Sequence import sequence
from directional_accuracy import calculate_directional_accuracy
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import praw
import threading
from threading import Lock
import requests
import datetime


stop_event = threading.Event()
lock = Lock()
analyzer = SentimentIntensityAnalyzer()

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

Scrape that motha fucka

Multithreading notes:

Plan is to run the reddit stream via PRAW in a seperate thread. Threads live inside of the same process and share processes memory.

see thread box for example.

to check if the thread is alive run t.is_alive() in a different box

to kill thread: stop_event.clear()

if you want the thread to run make sure stop_event.is_set() is on false




In [2]:
reddit = praw.Reddit(
    client_id="uClvvUAwSrTJgqogefWpvQ",
    client_secret="X3hGKPQjuCxU3O4yLKDjSE_ysdfVlg",
    username="Expensive-Deal3735",
    password="BatmanTheDarkWhite",
    user_agent="windows:SentimentBotScraper:0.1 (by u/Expensive-Deal3735)"
)
subreddits = reddit.subreddit("XRP")





In [17]:
BEARER_TOKEN = os.getenv("TWITTER_BEARER_TOKEN")
headers = {
    "Authorization": f"Bearer {BEARER_TOKEN}",
    "User-Agent": "XRP-scraper"
}
params = {
    "query": "#XRP",
    "max_results": 100,
    "tweet.fields": "id,text,created_at,public_metrics"
}
response = requests.get(
    "https://api.twitter.com/2/tweets/search/recent",
    headers=headers,
    params=params
)

if not response.ok:
    raise RuntimeError(f"Error {response.status_code}: {response.text}")
    
tweets = response.json().get("data", [])
print(f"Fetched {len(tweets)} tweets. Providing sentiment now:")







Fetched 100 tweets. Providing sentiment now:


In [25]:
cumm_sent = 0
for t in tweets:
    text = t["text"]
    vs = analyzer.polarity_scores(text)
    print(f" [{t['created_at']}] {text[:30]}... sentiment compound = {vs['compound']:.3f} \n")
    cumm_sent += vs['compound']
print(f"Cummulative Sentiment = {cumm_sent:.3f}")    

 [2025-07-18T19:14:25.000Z] ربح 127 دولار ✔️💰
💎 إشارات جوه... sentiment compound = 0.844 

 [2025-07-18T19:14:25.000Z] RT @johnnykrypto00: 🚨🚨#XRP at ... sentiment compound = 0.440 

 [2025-07-18T19:14:23.000Z] RT @RippleXrpie: 🚨BREAKING: Pr... sentiment compound = 0.000 

 [2025-07-18T19:14:23.000Z] 📛 investor report #Jetixbit is... sentiment compound = -0.382 

 [2025-07-18T19:14:21.000Z] RT @ParrotXrp: 🚀🇺🇸 HUGE NEWS f... sentiment compound = 0.712 

 [2025-07-18T19:14:17.000Z] RT @FrancoisDa31450: @bloodweb... sentiment compound = -0.468 

 [2025-07-18T19:14:16.000Z] #XRP https://t.co/2kHxF9mJoz... sentiment compound = 0.000 

 [2025-07-18T19:14:15.000Z] RT @Steph_iscrypto: 💥 BREAKING... sentiment compound = -0.821 

 [2025-07-18T19:14:14.000Z] RT @RippleXrpie: #XRP will des... sentiment compound = -0.542 

 [2025-07-18T19:14:14.000Z] What's your biggest lesson fro... sentiment compound = 0.000 

 [2025-07-18T19:14:13.000Z] RT @xtina_george: LIVE: Presid... sentiment compound = 0.000

Thread for continuous streaming:

In [ ]:


posts = []
currSentiment = 0
def streamCount():
    global currSentiment
    global posts
    for post in subreddits.stream.submissions(skip_existing=True):
        if stop_event.is_set():
            break
        
        posts.append(post.title + " " + post.selftext)
        with lock:
            vs = analyzer.polarity_scores(posts[-1])
            currSentiment += vs["compound"]


t = threading.Thread(target=streamCount, daemon=True)
t.start()

In [ ]:
#Checking thread status
#print(stop_event.is_set())
#pring(t.is_alive())
#stop_event.set()


False


Grab last 7 days of crpyto posts, for cryptomarkets and cryptocurrency well filter on title containing "XRP", then bucket into days, then grab avg sentiment on the 7 days, and we can compare with xrp price movement on those days

In [79]:
xrpPrices = yf.Ticker("XRP-USD")
data = xrpPrices.history(period="45d", interval = "1d")
#print(df.head())
data["Pct_change"] = data["Close"].pct_change()*100
movements = data["Pct_change"].dropna().tolist()
print(len(movements))
data.head()

44


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Pct_change
Date,,,,,,,,
2025-06-10 00:00:00+00:00,2.320374,2.324830,2.268008,2.305285,2753743744,0.0,0.0,NaN
2025-06-11 00:00:00+00:00,2.305316,2.335495,2.257220,2.270873,3016488242,0.0,0.0,-1.492740
2025-06-12 00:00:00+00:00,2.270873,2.271220,2.176238,2.194769,2658926570,0.0,0.0,-3.351317
2025-06-13 00:00:00+00:00,2.194732,2.194739,2.089059,2.149030,4448865758,0.0,0.0,-2.083998
2025-06-14 00:00:00+00:00,2.149034,2.178388,2.113941,2.141406,2001104713,0.0,0.0,-0.354761


In [120]:
xrpSubreddit = reddit.subreddit("XRP")
posts = xrpSubreddit.new(limit=500)
firstPost = next(posts)
currDate = datetime.datetime.utcfromtimestamp(firstPost.created_utc).date()
#note that we have to iterate backwards in time.
#takes the last 47 days
avgSentiment = []
currSentiment = 0
currCount = 0
for post in posts:
    
    if currDate != datetime.datetime.utcfromtimestamp(post.created_utc).date():
        if currCount > 0:
            avgSentiment.append(currSentiment / currCount)
            currSentiment = 0
            currCount = 0
            

    postText = post.title + " " + post.selftext
    currSentiment += analyzer.polarity_scores(postText)["compound"]
    currCount += 1
    currDate = datetime.datetime.utcfromtimestamp(post.created_utc).date()
    #print(currDate)




/tmp/ipykernel_1466/1281877119.py:4: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  currDate = datetime.datetime.utcfromtimestamp(firstPost.created_utc).date()
/tmp/ipykernel_1466/1281877119.py:12: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  if currDate != datetime.datetime.utcfromtimestamp(post.created_utc).date():
/tmp/ipykernel_1466/1281877119.py:22: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  currDate = datetime.datetim

In [ ]:

avgSentiment.reverse()
print(len(avgSentiment))
print(len(movements))



44
44


In [80]:
df = pd.DataFrame({
    'price_change': movements,
    'XRP_avg_sentiment': avgSentiment
})

for lag in range(1,6):
    df[f'sentiment_lag {lag}'] = df['XRP_avg_sentiment'].shift(lag)

for lag in range(1,6):
    df[f'price_change lag {lag}'] = df['price_change'].shift(lag)

#dropNA
df = df.dropna().reset_index(drop=True)

for lag in range (1,6):
    
    print(f"Pearson correlation for lag {lag} = {df['XRP_avg_sentiment'].corr(df[f'price_change lag {lag}'])}")
#print(f"Pearson correlation: {corr_value:.3f}")



Pearson correlation for lag 1 = 0.13934110464926797
Pearson correlation for lag 2 = 0.2952665961544778
Pearson correlation for lag 3 = 0.16652428674040726
Pearson correlation for lag 4 = -0.010319125729922184
Pearson correlation for lag 5 = -0.11248096252791245
